Road density is one of the hardest covariates to harvest, as there exist no online dataset available and as such the data has to be, albeit fairly easily, derived from the OpenStreetMap site. 
This notebook uses OSM's Python API osmnx to get the data we need.

In [1]:
import pandas as pd
import osmnx as ox
from pyquadkey2 import quadkey
from pyquadkey2.quadkey import QuadKey
from pyquadkey2.quadkey import TileAnchor
from shapely.geometry import box

Due to the enormous amount of computational time used to get the road data, it's important to limit the harvesting to the sole tiles of interest of the study. 
Since osmnx retrieves a geodataframe of a given area, to reduce the number of access we aim to request data from a bigger area, and then work on the proper quadkey 14 tiles as subsets of the geodataframe. The easier method to assign a bigger area to each quadkey is simply to have it be a parent quadkey of a lower level. We choose the parent level to be 9 as its the biggest area to not put a strain on the hard drive. The relation parent-child tiles is stored in a dictionary of 9962 keys.

In [2]:
with open('quad_paesi2.txt', 'r') as file:
    list_quads = file.read()
    list_quads = list_quads.split(',')

In [3]:
diz = {}

for quad in list_quads:
    parent = quadkey.from_str(quad).parent().parent().parent().parent().parent()
    if parent not in diz.keys():
        diz[parent] = [quad]
    else:
        diz[parent].append(quad)

In [4]:
len(diz.keys())

9962

In [5]:
len(list_quads)

3839219

We're now going to illustrate how the process work for a single quadkey 9 tile area: using pyquadkey2's to_geo() function we get the boundings of the area and feed it to osmnx's graph_from_bbox, which retrieves a graph diagram of nodes and edges representing the road network of the area, with the edges being the streets. We specify the network type to include only roads where its allowed to drive a vehicle in.
The next function, graph_to_gdfs creates a geodataframe from the graph diagram, excluding the nodes as they are no interest to us. As we can see, the retrieval process time is reasonable, albeit pretty long.
We replicated the process twice, one without using a caching system and one using it; since each tile 14 is strictly bound to its tile 9 parent, once an iteration is completed there is no need to reuse the data, so not caching is a reasonable action, especially after witnessing it not resulting in an increased runtime.

In [6]:
parent = list(diz.keys())[1]
parent

300122333

In [7]:
n,w = parent.to_geo()
s,e = parent.to_geo(anchor = TileAnchor.ANCHOR_SE)

(n,s,e,w)

(-21.28937435586, -21.943045533438, 28.125, 27.421875)

In [8]:
import time
start = time.time()
ox.settings.use_cache = False
gdf_edges = ox.graph_from_bbox(bbox = (n,s,e,w), network_type='drive')
gdf_edges = ox.graph_to_gdfs(gdf_edges, nodes=False)
end = time.time()
print(f'elapsed time: {end-start}s')

elapsed time: 16.33259129524231s


In [9]:
import time
start = time.time()
ox.settings.use_cache = True
gdf_edges = ox.graph_from_bbox(bbox = (n,s,e,w), network_type='drive')
gdf_edges = ox.graph_to_gdfs(gdf_edges, nodes=False)
end = time.time()
print(f'elapsed time: {end-start}s')

elapsed time: 17.238303422927856s


In [10]:
len(gdf_edges)

19104

In [11]:
len(diz[parent])

544

Once the geodataframe is ready, we can get the road density data for all the child tiles 14. The operation is easy: after assigning to each tile 14 its bounding box, a subset of roads only intersecting the smaller area is taken. The road density is then calculated as the ratio between the total length of the roads (in kilometers) and the area (in squared kilometers. Since the quadkey object as a built-in function to retrieve its area, we can take a shortcut from calculating it properly from its boundaries.). The whole operation is very fast, taking only 10 seconds to calculate the road density for all 544 child tiles.

In [12]:
import warnings
df = []
i = 0
start = time.time()
for child in diz[parent]:
    i +=1
    warnings.simplefilter("ignore")
    q = quadkey.from_str(child)
    no,we = q.to_geo()
    so,ea = q.to_geo(anchor = TileAnchor.ANCHOR_SE)
    area = q.area() / 1000000
    bbox = box(we,so,ea,no)
    rd_length = (gdf_edges[gdf_edges.intersects(bbox)].length.sum())/1000
    df.append({'quadkey': child, 'road_density': rd_length/area})
    if i%50 == 0:
        print(i)
end = time.time()
print(f'elapsed time: {end-start}s')

50
100
150
200
250
300
350
400
450
500
elapsed time: 10.459198713302612s


In [13]:
df

[{'quadkey': '30012233321231', 'road_density': 0.0},
 {'quadkey': '30012233302122', 'road_density': 1.8483818028690114e-05},
 {'quadkey': '30012233302121', 'road_density': 1.768249738680087e-05},
 {'quadkey': '30012233323212', 'road_density': 0.0},
 {'quadkey': '30012233313321', 'road_density': 0.0},
 {'quadkey': '30012233322013', 'road_density': 0.0},
 {'quadkey': '30012233300023', 'road_density': 8.545735808112624e-05},
 {'quadkey': '30012233302123', 'road_density': 5.7448544786898545e-05},
 {'quadkey': '30012233323231', 'road_density': 0.0},
 {'quadkey': '30012233303310', 'road_density': 0.0},
 {'quadkey': '30012233333211', 'road_density': 0.0},
 {'quadkey': '30012233302320', 'road_density': 0.0},
 {'quadkey': '30012233303321', 'road_density': 3.8100591864276994e-05},
 {'quadkey': '30012233320132', 'road_density': 0.0},
 {'quadkey': '30012233300203', 'road_density': 0.00012241011207257481},
 {'quadkey': '30012233322222', 'road_density': 0.0},
 {'quadkey': '30012233302333', 'road_den

In [14]:
data = pd.DataFrame(df)
data

,quadkey,road_density
0,30012233321231,0.000000
1,30012233302122,0.000018
2,30012233302121,0.000018
3,30012233323212,0.000000
4,30012233313321,0.000000
...,...,...
539,30012233301111,0.000000
540,30012233330033,0.000000
541,30012233331023,0.000000
542,30012233303211,0.000000


In [15]:
data.road_density.describe()

count    544.000000
mean       0.000019
std        0.000038
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000027
max        0.000250
Name: road_density, dtype: float64

To better automate the process, we create the functions dens_from_quadkey(); which takes in input a quadkey and a geodataframe, and returns its road density; and get_road_data(); which inputs a quadkey and returns a geodataframe containing all the roads of the area. Since there happens to be quadkeys were no road data is available, the function graph_from_bbox could fail. In that case, we can assume that no road is present in the area and all of the child tiles will have no roads and thus no density.

In [16]:
child = diz[parent][0]
child

'30012233321231'

In [17]:
no,we = quadkey.from_str(child).to_geo()
so,ea = quadkey.from_str(child).to_geo(anchor = TileAnchor.ANCHOR_SE)

In [4]:
def dens_from_quadkey(quad: str, data):
    '''
    Get the road density for a given quadkey.

    Args:
    quad (str): quadkey identifier as string value
    data (gdf): pandas geodataframe containing all roads in a given bigger area than the required quadkey
    
    Returns:
    road_density (float)
    '''
    q = quadkey.from_str(quad)
    n,w = q.to_geo()
    s,e = q.to_geo(anchor = TileAnchor.ANCHOR_SE)
    
    ##area/10^6 since its supposed to be in km^2
    area = q.area() / 1000000  
    bbox = box(w,s,e,n)

    ##rd_length/1000 since its supposed to be in km
    rd_length = (data[data.intersects(bbox)].length.sum())/1000

    return rd_length/area

In [5]:
def get_road_data(quad):
    '''
    Gets the road data from OpenStreetMap for a given, generally low res quadkey

    Args:
    quad (quadkey): quadkey

    Returns:
    gdf_edges (gdf): pandas geodataframe containing all roads for the given area.
                     it has only length and geometry values as to render the following 
                     slicing process quicker
    '''
    n,w = parent.to_geo()
    s,e = parent.to_geo(anchor = TileAnchor.ANCHOR_SE)
    succ = False
    try: 
        gdf_edges = ox.graph_from_bbox(bbox = (n,s,e,w), network_type='drive')
        if gdf_edges is None:
            raise Exception("La funzione secondaria ha fallito")
        gdf_edges = ox.graph_to_gdfs(gdf_edges, nodes=False)
        succ = True

    except Exception as e:
        # Se la funzione secondaria fallisce, gestisci l'eccezione
        #print("Errore:", e)
        
        # Assegna un output arbitrario
        output = []

    if succ == True:
        return gdf_edges[['length', 'geometry']]
    else:
        return output

As we can see, not caching should only cost around 1 second of additional runtime for each tile 9.

In [20]:
df = []
i = 0
ox.settings.use_cache = False
start = time.time()
for parent, childlist in diz.items():
    gdf_edges = get_road_data(parent)

    if len(gdf_edges) == 0:
        for child in childlist:
            df.append({'quadkey': child, 'road_density': 0})
    else:
        for child in childlist:
            dens = dens_from_quadkey(child, gdf_edges)
            df.append({'quadkey': child, 'road_density': dens})
    i += 1
    print(i)
    if i == 3:
        break

end = time.time()
print(f'elapsed time: {end-start}s')

1
2
3
elapsed time: 88.36677837371826s


In [21]:
df = []
i = 0
ox.settings.use_cache = True
start = time.time()
for parent, childlist in diz.items():
    gdf_edges = get_road_data(parent)

    if len(gdf_edges) == 0:
        for child in childlist:
            df.append({'quadkey': child, 'road_density': 0})
    else:
        for child in childlist:
            dens = dens_from_quadkey(child, gdf_edges)
            df.append({'quadkey': child, 'road_density': dens})
    i += 1
    print(i)
    if i == 3:
        break

end = time.time()
print(f'elapsed time: {end-start}s')

1
2
3
elapsed time: 85.31292819976807s


In [22]:
df = pd.DataFrame(df)
len(df)

2278

In [23]:
df.describe()

,road_density
count,2278.000000
mean,0.000014
std,0.000027
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000021
max,0.000254


In [24]:
len(df[df.road_density == 0])

1524

Due to the high runtime, the operation was subsetted into multiple machines; this single machine only got the first 1245 tile 9s; either way, below its illustrated how the work had been divided.

In [25]:
len(diz.keys())

9962

In [26]:
9962//2

4981

The list of quadkey 9 tiles is split; then we decide an arbitrary number of batches and work with each batch.
Due to time constraints, even the first (and only) batch treated with this machine had to be split into multiple datasets, but its concatenation is trivial and had no impact on the work. The whole process took around 10 hours to retrieve 855k quadkey 14 tiles.

In [6]:
lista_uso = list(diz.keys())[:4981]

In [7]:
import numpy as np
steps = np.linspace(0, 4981, 5)
steps = [int(s) for s in steps]
steps

[0, 1245, 2490, 3735, 4981]

In [9]:
import warnings
df = []
i = 536

for parent in lista_uso[536:steps[1]]:
    warnings.simplefilter("ignore")
    gdf_edges = get_road_data(parent)
    childlist = diz[parent]

    if len(gdf_edges) == 0:
        for child in childlist:
            df.append({'quadkey': child, 'road_density': 0})
    else:
        for child in childlist:
            dens = dens_from_quadkey(child, gdf_edges)
            df.append({'quadkey': child, 'road_density': dens})
    i += 1

In [10]:
df = pd.DataFrame(df)
df

,quadkey,road_density
0,12213232222133,1.471658e-05
1,12213232220210,0.000000e+00
2,12213232221202,9.930837e-05
3,12213232222131,6.784457e-05
4,12213232220303,6.656877e-05
...,...,...
213866,21003130231002,7.408945e-06
213867,21003130212212,7.374538e-05
213868,21003130211230,8.953491e-07
213869,21003130221130,6.507240e-05


In [11]:
cart = "C:\\Users\\Luca\\Downloads\\RWI\\road_dens"
df = df.groupby('quadkey', as_index = False).mean()
df.to_csv(cart + '\\road_density_pt4.csv', index = False)

In [11]:
import time
import warnings
start = time.time()
path2 = "C:\\Users\\Luca\\Downloads\\RWI\\road_dens\\road_density_pt3.csv"

# Scrivere le righe
i = 712
ox.settings.use_cache = False
df = []
for parent in lista_uso[712:steps[1]]:
    warnings.simplefilter("ignore")
    gdf_edges = get_road_data(parent)
    #print('geodf ready')
    childlist = diz[parent]
    
    if len(gdf_edges) == 0:
        for child in childlist:
            df.append({'quadkey': child, 'road_density': 0})
    else:
        for child in childlist:
            dens = dens_from_quadkey(child, gdf_edges)
            df.append({'quadkey': child, 'road_density': dens})
    i += 1
    if i%25 == 0:
        print(i)

df = pd.DataFrame(df)
print(len(df))
df.to_csv(path2, index = False)
end = time.time()
print(f'elapsed time: {end-start}s')

725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
356077
elapsed time: 21200.815188884735s


In [13]:
usati = []

for parent in lista_uso[:steps[1]]:
    usati += diz[parent]

len(usati)

855527

In [15]:
import os
cart = "C:\\Users\\Luca\\Downloads\\RWI\\road_dens"

newdf = pd.DataFrame(columns = ['quadkey', 'road_density'])

for file in os.listdir(cart):
    temp = pd.read_csv(f'{cart}\\{file}')

    print(f'{file}: {temp.shape}')
    newdf = pd.concat([newdf, temp], ignore_index = True)

newdf

road_density_pt1.csv: (47375, 2)
road_density_pt2.csv: (452556, 2)
road_density_pt3.csv: (356077, 2)


,quadkey,road_density
0,3311123010003,0.0
1,3311123000100,0.0
2,3311123001210,0.0
3,3311123011311,0.0
4,3311123021220,0.0
...,...,...
856003,3331212110021,0.0
856004,3331212131311,0.0
856005,3331212101110,0.0
856006,3331212103132,0.0


In [16]:
newdf.describe()

,road_density
count,856008.000000
mean,0.000021
std,0.000031
min,0.000000
25%,0.000000
50%,0.000009
75%,0.000035
max,0.000522


In [20]:
newdf = newdf.astype({'quadkey': 'str'})

In [23]:
newdf.drop_duplicates(inplace = True)

In [24]:
len(newdf[newdf.quadkey.isin(usati)])

855527

In [25]:
len(newdf[newdf.road_density > 0])

451441

In [27]:
newdf[newdf.road_density > 0].road_density.min()

4.4882091515504944e-08

In [28]:
newdf[newdf.road_density > 0].min()

quadkey         10223223102333
road_density               0.0
dtype: object

In [30]:
newdf.to_csv(f'{cart}\\road_density_pt4.csv', index = False)

Below is shown the concatenation of the entire road density data.

In [3]:
import pandas as pd
import os

repo = "C:\\Users\\Luca\\Downloads\\ROAD DENSITY"

df = pd.read_csv(f'{repo}\\{os.listdir(repo)[0]}')

for file in os.listdir(repo)[1:]:
    print(file[-5])
    temp = pd.read_csv(f'{repo}\\{file}')
    print(temp.shape)

    df = pd.concat([df, temp], ignore_index = True)
    print(df.shape)

df

1
(549793, 2)
(1118397, 2)
0
(855527, 2)
(1973924, 2)
2
(509783, 2)
(2483707, 2)
3
(444355, 2)
(2928062, 2)
4
(383490, 2)
(3311552, 2)
5
(272483, 2)
(3584035, 2)
6
(147679, 2)
(3731714, 2)
7
(70126, 2)
(3801840, 2)
8
(30173, 2)
(3832013, 2)
9
(7205, 2)
(3839218, 2)


,quadkey,road_density
0,12220322013131,0.000000
1,12220322012313,0.000018
2,12220322032322,0.000014
3,12220322033312,0.000000
4,12220322032013,0.000035
...,...,...
3839213,12220113313330,0.000000
3839214,13222110100033,0.000000
3839215,12220132300200,0.000016
3839216,3331030333230,0.000000


The check guarantees that all the quadkeys required for the study are accounted for.

In [4]:
with open('quad_paesi2.txt', 'r') as file:
    list_quads = file.read()
    list_quads = list_quads.split(',')

In [5]:
df = df.astype({'quadkey': 'str'})

In [6]:
len(list_quads)

3839219

In [7]:
len(df[df.quadkey.isin(list_quads)])

3839218

In [8]:
len(df.quadkey.unique())

3839218

In [9]:
df.describe()

,road_density
count,3.839218e+06
mean,2.130730e-05
std,3.158068e-05
min,0.000000e+00
25%,0.000000e+00
50%,6.705920e-06
75%,3.431291e-05
max,6.689970e-04


In [10]:
df.to_csv("C:\\Users\\Luca\\Downloads\\RWI\\road_dens\\road_density_quadkey_utili.csv", index = False)